## Lesson 15 - Pandas Time Series

*Adapted from Pandas documentation (esoteric/financial applications left out).*

In working with time series data, we will frequently seek to:

* generate sequences of fixed-frequency dates and time spans
* conform or convert time series to a particular frequency
* compute “relative” dates based on various non-standard time increments (e.g. 5 business days before the last business day of the year), or “roll” dates forward or backward

Pandas provides a relatively compact and self-contained set of tools for performing the above tasks.

In [1]:
# modules used in this tutorial
import pandas as pd
import numpy as np
import pytz
import dateutil

In [2]:
# set max row and random seed
pd.set_option("display.max_rows", 20)
np.random.seed(12)

## Examples

Create a range of dates:

In [3]:
rng = pd.date_range('1/1/2011', periods=72, freq='H')

In [4]:
rng[:5]

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 01:00:00',
               '2011-01-01 02:00:00', '2011-01-01 03:00:00',
               '2011-01-01 04:00:00'],
              dtype='datetime64[ns]', freq='H')

In [5]:
len(rng)

72

Index Pandas objects with dates:

In [6]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [7]:
ts.head()

2011-01-01 00:00:00    0.472986
2011-01-01 01:00:00   -0.681426
2011-01-01 02:00:00    0.242439
2011-01-01 03:00:00   -1.700736
2011-01-01 04:00:00    0.753143
Freq: H, dtype: float64

Change frequency and fill gaps:

In [8]:
# to 45 minute frequency and forward fill
converted = ts.asfreq('45Min', method='pad')

In [9]:
converted.head()

2011-01-01 00:00:00    0.472986
2011-01-01 00:45:00    0.472986
2011-01-01 01:30:00   -0.681426
2011-01-01 02:15:00    0.242439
2011-01-01 03:00:00   -1.700736
Freq: 45T, dtype: float64

Frequency conversion and resampling of a time series:

In [10]:
ts.resample('D', how='mean')

/Users/luke.thompson/miniconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  """Entry point for launching an IPython kernel.


2011-01-01    0.106866
2011-01-02   -0.449748
2011-01-03    0.006654
Freq: D, dtype: float64

In [11]:
ts.resample('D').mean()

2011-01-01    0.106866
2011-01-02   -0.449748
2011-01-03    0.006654
Freq: D, dtype: float64

In [12]:
ts.resample('T').mean().dropna()

2011-01-01 00:00:00    0.472986
2011-01-01 01:00:00   -0.681426
2011-01-01 02:00:00    0.242439
2011-01-01 03:00:00   -1.700736
2011-01-01 04:00:00    0.753143
2011-01-01 05:00:00   -1.534721
2011-01-01 06:00:00    0.005127
2011-01-01 07:00:00   -0.120228
2011-01-01 08:00:00   -0.806982
2011-01-01 09:00:00    2.871819
                         ...   
2011-01-03 14:00:00   -0.083449
2011-01-03 15:00:00    0.677806
2011-01-03 16:00:00    0.556060
2011-01-03 17:00:00    0.222719
2011-01-03 18:00:00   -1.528985
2011-01-03 19:00:00    1.029211
2011-01-03 20:00:00   -1.166259
2011-01-03 21:00:00   -1.009562
2011-01-03 22:00:00   -0.105268
2011-01-03 23:00:00    0.512022
Length: 72, dtype: float64

## Overview

The following table shows the type of time-related classes pandas can handle and how to create them:

Class | Remarks | How to create
------|---------|--------------
`Timestamp` | Represents a single time stamp | `to_datetime`, `Timestamp`
`DatetimeIndex` | Index (~array) of `Timestamp` | `to_datetime`, `date_range`, `DatetimeIndex`
`Period` | Represents a single time span | `Period`
`PeriodIndex` | Index (~array) of `Period` | `period_range`, `PeriodIndex`

## Timestamps vs. Periods (time spans)

Time-stamped data is the most basic type of timeseries data that associates values with points in time. For pandas objects it means using the points in time.

In [13]:
pd.datetime(2012, 5, 1)

datetime.datetime(2012, 5, 1, 0, 0)

In [14]:
pd.Timestamp(pd.datetime(2012, 5, 1))

Timestamp('2012-05-01 00:00:00')

In [15]:
pd.Timestamp('2012-05-01')

Timestamp('2012-05-01 00:00:00')

However, in many cases it is more natural to associate things like change variables with a time span instead. The span represented by Period can be specified explicitly, or inferred from datetime string format.

For example:

In [16]:
pd.Period('2011-01')

Period('2011-01', 'M')

In [17]:
pd.Period('2012-05', freq='D')

Period('2012-05-01', 'D')

Timestamp and Period can be the index. Lists of Timestamp and Period are automatically coerced to DatetimeIndex and PeriodIndex respectively.

In [18]:
dates = [pd.Timestamp('2012-05-01'), pd.Timestamp('2012-05-02'), pd.Timestamp('2012-05-03')]

In [19]:
ts = pd.Series(np.random.randn(3), dates)

In [20]:
ts

2012-05-01    1.407728
2012-05-02   -1.687696
2012-05-03    1.471234
dtype: float64

In [21]:
ts.index

DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

In [22]:
type(ts.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [23]:
periods = [pd.Period('2012-01'), pd.Period('2012-02'), pd.Period('2012-03')]

In [24]:
ts = pd.Series(np.random.randn(3), periods)

In [25]:
ts

2012-01    1.636463
2012-02   -0.461395
2012-03   -0.201362
Freq: M, dtype: float64

In [26]:
ts.index

PeriodIndex(['2012-01', '2012-02', '2012-03'], dtype='period[M]', freq='M')

In [27]:
type(ts.index)

pandas.core.indexes.period.PeriodIndex

Pandas allows you to capture both representations and convert between them. Under the hood, Pandas represents timestamps using instances of Timestamp and sequences of timestamps using instances of DatetimeIndex. For regular time spans, Pandas uses Period objects for scalar values and PeriodIndex for sequences of spans.

## Converting to Timestamps

To convert a Series or list-like object of date-like objects e.g. strings, epochs, or a mixture, you can use the to_datetime function. When passed a Series, this returns a Series (with the same index), while a list-like is converted to a DatetimeIndex:

In [28]:
pd.to_datetime(pd.Series(['Jul 31, 2009', '2010-01-10', None]))

0   2009-07-31
1   2010-01-10
2          NaT
dtype: datetime64[ns]

In [29]:
pd.to_datetime(['2005/11/23', '2010.12.31'])

DatetimeIndex(['2005-11-23', '2010-12-31'], dtype='datetime64[ns]', freq=None)

If you use dates which start with the day first (i.e. European style), you can pass the dayfirst flag.

**Warning:** You see in the second example below that dayfirst isn’t strict, so if a date can’t be parsed with the day being first it will be parsed as if dayfirst were False.

In [30]:
pd.to_datetime(['04-01-2012 10:00'], dayfirst=True)

DatetimeIndex(['2012-01-04 10:00:00'], dtype='datetime64[ns]', freq=None)

In [31]:
pd.to_datetime(['04-01-2012 10:00'], dayfirst=False)

DatetimeIndex(['2012-04-01 10:00:00'], dtype='datetime64[ns]', freq=None)

In [32]:
# default behavior is dayfirst=False
pd.to_datetime(['04-01-2012 10:00'])

DatetimeIndex(['2012-04-01 10:00:00'], dtype='datetime64[ns]', freq=None)

In [33]:
pd.to_datetime(['14-01-2012', '01-14-2012'], dayfirst=True)

DatetimeIndex(['2012-01-14', '2012-01-14'], dtype='datetime64[ns]', freq=None)

**Note:** Specifying a format argument will potentially speed up the conversion considerably and on versions later than 0.13.0 explicitly specifying a format string of ‘%Y%m%d’ takes a faster path still.

If you pass a single string to to_datetime, it returns single Timestamp. Also, Timestamp can accept the string input. Note that Timestamp doesn’t accept string parsing option like dayfirst or format: use to_datetime if these are required.

The first option below would work with a list of dates, but the second would not.

In [34]:
#pd.to_datetime('11/12/2010', format='%m/%d/%Y')
pd.to_datetime('11/12/2010', format='%d/%m/%Y')

Timestamp('2010-12-11 00:00:00')

In [35]:
pd.to_datetime('11/12/2010')

Timestamp('2010-11-12 00:00:00')

In [36]:
pd.Timestamp('2010-11-12')

Timestamp('2010-11-12 00:00:00')

### Invalid data

In version 0.17.0, the default for to_datetime is now errors='raise', rather than errors='ignore'. This means that invalid parsing will raise rather that return the original input as in previous versions.

Pass errors='coerce' to convert invalid data to NaT (not a time):

In [37]:
# don't convert anything and return the original input when unparseable
pd.to_datetime(['2009/07/31', 'asd'], errors='ignore')

array(['2009/07/31', 'asd'], dtype=object)

In [38]:
# return NaT for input when unparseable
pd.to_datetime(['2009/07/31', 'asd'], errors='coerce')

DatetimeIndex(['2009-07-31', 'NaT'], dtype='datetime64[ns]', freq=None)

### Epoch timestamps

It’s also possible to convert integer or float epoch times. The default unit for these is nanoseconds (since these are how Timestamps are stored). However, often epochs are stored in another unit which can be specified:

Typical epoch stored units (nanoseconds):

In [39]:
pd.to_datetime(0)

Timestamp('1970-01-01 00:00:00')

In [40]:
pd.to_datetime(1)

Timestamp('1970-01-01 00:00:00.000000001')

In [41]:
pd.to_datetime(3e9)

Timestamp('1970-01-01 00:00:03')

In [42]:
pd.to_datetime([1349720105, 1349806505, 1349892905, 1349979305, 1350065705], unit='s')

DatetimeIndex(['2012-10-08 18:15:05', '2012-10-09 18:15:05',
               '2012-10-10 18:15:05', '2012-10-11 18:15:05',
               '2012-10-12 18:15:05'],
              dtype='datetime64[ns]', freq=None)

In [43]:
pd.to_datetime([1349720105100, 1349720105200, 1349720105300, 1349720105400, 1349720105500 ], unit='ms')

DatetimeIndex(['2012-10-08 18:15:05.100000', '2012-10-08 18:15:05.200000',
               '2012-10-08 18:15:05.300000', '2012-10-08 18:15:05.400000',
               '2012-10-08 18:15:05.500000'],
              dtype='datetime64[ns]', freq=None)

**Note:** Epoch times will be rounded to the nearest nanosecond.

## Generating ranges of timestamps

To generate an index with time stamps, you can use either the DatetimeIndex or Index constructor and pass in a list of datetime objects

### The slow way

In [44]:
dates = [pd.datetime(2012, 5, 1), pd.datetime(2012, 5, 2), pd.datetime(2012, 5, 3)]

In [45]:
index = pd.DatetimeIndex(dates)

In [46]:
index # Note the frequency information

DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

In [47]:
index = pd.Index(dates)

In [48]:
index # Automatically converted to DatetimeIndex

DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

### The quick way - data_range by periods or start-end

Practically, this becomes very cumbersome because we often need a very long index with a large number of timestamps. 

If we need timestamps on a regular frequency, we can use the pandas function date_range to create timestamp indexes. (A similar function and bdate_range does the same thing for business days; the 'b' stands for 'business'.)

Information needed to specify date ranges:

* start, number of periods, AND frequency
* start, end, AND frequency

Frequency defaults to 'D' for date_range and 'B' for bdate_range.

In [49]:
# freq='M' uses last day of month by default
index = pd.date_range('2018-3-1', periods=1000, freq='M')
index 

DatetimeIndex(['2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30',
               '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31',
               '2018-11-30', '2018-12-31',
               ...
               '2100-09-30', '2100-10-31', '2100-11-30', '2100-12-31',
               '2101-01-31', '2101-02-28', '2101-03-31', '2101-04-30',
               '2101-05-31', '2101-06-30'],
              dtype='datetime64[ns]', length=1000, freq='M')

In [50]:
# freq='B' is business days
index = pd.date_range('2018-3-1', periods=1000, freq='B')
index

DatetimeIndex(['2018-03-01', '2018-03-02', '2018-03-05', '2018-03-06',
               '2018-03-07', '2018-03-08', '2018-03-09', '2018-03-12',
               '2018-03-13', '2018-03-14',
               ...
               '2021-12-16', '2021-12-17', '2021-12-20', '2021-12-21',
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-27',
               '2021-12-28', '2021-12-29'],
              dtype='datetime64[ns]', length=1000, freq='B')

In [51]:
# default is freq='D' for date_range
index = pd.date_range('2018-3-1', periods=1000)
index

DatetimeIndex(['2018-03-01', '2018-03-02', '2018-03-03', '2018-03-04',
               '2018-03-05', '2018-03-06', '2018-03-07', '2018-03-08',
               '2018-03-09', '2018-03-10',
               ...
               '2020-11-15', '2020-11-16', '2020-11-17', '2020-11-18',
               '2020-11-19', '2020-11-20', '2020-11-21', '2020-11-22',
               '2020-11-23', '2020-11-24'],
              dtype='datetime64[ns]', length=1000, freq='D')

In [52]:
# default is freq='B' for bdate_range
index = pd.bdate_range('2018-3-1', periods=1000)
index

DatetimeIndex(['2018-03-01', '2018-03-02', '2018-03-05', '2018-03-06',
               '2018-03-07', '2018-03-08', '2018-03-09', '2018-03-12',
               '2018-03-13', '2018-03-14',
               ...
               '2021-12-16', '2021-12-17', '2021-12-20', '2021-12-21',
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-27',
               '2021-12-28', '2021-12-29'],
              dtype='datetime64[ns]', length=1000, freq='B')

### Commonly used of offset aliases

Alias | Description
------|------------
D | day frequency
B | business day frequency
W | weekly frequency
BW | business week end frequency
M | month end frequency
BM | business month end frequency
MS | month start frequency
A | year end frequency
BA | business year end frequency
AS | year start frequency
H | hourly frequency
T, min | minutely frequency
S | secondly frequency
L, ms | milliseonds
U, us | microseconds
N | nanoseconds

In [53]:
t1 = pd.datetime(2011, 1, 1)
t2 = pd.datetime(2012, 1, 1)

In [54]:
rng = pd.date_range(t1, t2, freq='D')
rng

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03', '2011-01-04',
               '2011-01-05', '2011-01-06', '2011-01-07', '2011-01-08',
               '2011-01-09', '2011-01-10',
               ...
               '2011-12-23', '2011-12-24', '2011-12-25', '2011-12-26',
               '2011-12-27', '2011-12-28', '2011-12-29', '2011-12-30',
               '2011-12-31', '2012-01-01'],
              dtype='datetime64[ns]', length=366, freq='D')

In [55]:
pd.date_range(t1, t2, freq='W-SUN')

DatetimeIndex(['2011-01-02', '2011-01-09', '2011-01-16', '2011-01-23',
               '2011-01-30', '2011-02-06', '2011-02-13', '2011-02-20',
               '2011-02-27', '2011-03-06', '2011-03-13', '2011-03-20',
               '2011-03-27', '2011-04-03', '2011-04-10', '2011-04-17',
               '2011-04-24', '2011-05-01', '2011-05-08', '2011-05-15',
               '2011-05-22', '2011-05-29', '2011-06-05', '2011-06-12',
               '2011-06-19', '2011-06-26', '2011-07-03', '2011-07-10',
               '2011-07-17', '2011-07-24', '2011-07-31', '2011-08-07',
               '2011-08-14', '2011-08-21', '2011-08-28', '2011-09-04',
               '2011-09-11', '2011-09-18', '2011-09-25', '2011-10-02',
               '2011-10-09', '2011-10-16', '2011-10-23', '2011-10-30',
               '2011-11-06', '2011-11-13', '2011-11-20', '2011-11-27',
               '2011-12-04', '2011-12-11', '2011-12-18', '2011-12-25',
               '2012-01-01'],
              dtype='datetime64[ns]', freq='W-S

In [56]:
pd.date_range(t1, t2, freq='B')

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14',
               ...
               '2011-12-19', '2011-12-20', '2011-12-21', '2011-12-22',
               '2011-12-23', '2011-12-26', '2011-12-27', '2011-12-28',
               '2011-12-29', '2011-12-30'],
              dtype='datetime64[ns]', length=260, freq='B')

In [57]:
pd.date_range(t1, t2, freq='BM')

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31', '2011-06-30', '2011-07-29', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-30'],
              dtype='datetime64[ns]', freq='BM')

In [58]:
pd.bdate_range(end=t2, periods=20)

DatetimeIndex(['2011-12-05', '2011-12-06', '2011-12-07', '2011-12-08',
               '2011-12-09', '2011-12-12', '2011-12-13', '2011-12-14',
               '2011-12-15', '2011-12-16', '2011-12-19', '2011-12-20',
               '2011-12-21', '2011-12-22', '2011-12-23', '2011-12-26',
               '2011-12-27', '2011-12-28', '2011-12-29', '2011-12-30'],
              dtype='datetime64[ns]', freq='B')

In [59]:
pd.bdate_range(start=t1, periods=20)

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14', '2011-01-17', '2011-01-18',
               '2011-01-19', '2011-01-20', '2011-01-21', '2011-01-24',
               '2011-01-25', '2011-01-26', '2011-01-27', '2011-01-28'],
              dtype='datetime64[ns]', freq='B')

## DatetimeIndex

One of the main uses for DatetimeIndex is as an index for Pandas objects. The DatetimeIndex class contains many timeseries related optimizations:

* A large range of dates for various offsets are pre-computed and cached under the hood in order to make generating subsequent date ranges very fast (just have to grab a slice)
* Fast shifting using the shift and tshift method on pandas objects
* Unioning of overlapping DatetimeIndex objects with the same frequency is very fast (important for fast data alignment)
* Quick access to date fields via properties such as year, month, etc.
* Regularization functions like snap and very fast asof logic

DatetimeIndex objects has all the basic functionality of regular Index objects and a smorgasbord of advanced timeseries-specific methods for easy frequency processing.

See also documentation for *Reindexing methods*.

**Note:** While Pandas does not force you to have a sorted date index, some of these methods may have unexpected or incorrect behavior if the dates are unsorted. So please be careful.

DatetimeIndex can be used like a regular index and offers all of its intelligent functionality like selection, slicing, etc.

In [60]:
start = pd.datetime(2011, 1, 1)
end = pd.datetime(2012, 1, 1)

In [61]:
rng = pd.date_range(start, end, freq='BM')

In [62]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [63]:
ts.index

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31', '2011-06-30', '2011-07-29', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-30'],
              dtype='datetime64[ns]', freq='BM')

In [64]:
ts[:5].index

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31'],
              dtype='datetime64[ns]', freq='BM')

In [65]:
ts[::2].index

DatetimeIndex(['2011-01-31', '2011-03-31', '2011-05-31', '2011-07-29',
               '2011-09-30', '2011-11-30'],
              dtype='datetime64[ns]', freq='2BM')

### Slicing revisited

In Pandas, **label-based slicing is inclusive**, whereas **index-based slicing is NOT inclusive** (like index-based slicing in Python generally).

In [66]:
x = pd.Series([0.0, 0.1, 0.2], index=['a', 'b', 'c'])

In [67]:
x[0:2]

a    0.0
b    0.1
dtype: float64

In [68]:
x['a':'c']

a    0.0
b    0.1
c    0.2
dtype: float64

DatetimeIndex-based slicing is effectively label-based, so it is inclusive. It will not generate any dates outside of those dates if specified.

### DatetimeIndex partial string 'smart' indexing

You can pass in dates and strings that parse to dates as indexing parameters:

In [69]:
ts['1/31/2011']

-0.5718167302823739

In [70]:
ts[pd.datetime(2011, 12, 25):]

2011-12-30    0.356223
Freq: BM, dtype: float64

In [71]:
ts['10/31/2011':'12/31/2011']

2011-10-31    0.146560
2011-11-30   -0.466351
2011-12-30    0.356223
Freq: BM, dtype: float64

To provide convenience for accessing longer time series, you can also pass in the year or year and month as strings:

In [72]:
ts['2011']

2011-01-31   -0.571817
2011-02-28   -0.603299
2011-03-31   -1.339389
2011-04-29   -1.689653
2011-05-31   -0.199327
2011-06-30    0.257773
2011-07-29    1.828821
2011-08-31   -1.001002
2011-09-30   -2.091691
2011-10-31    0.146560
2011-11-30   -0.466351
2011-12-30    0.356223
Freq: BM, dtype: float64

In [73]:
ts['2011-6']

2011-06-30    0.257773
Freq: BM, dtype: float64

This type of slicing will work on a DataFrame with a DateTimeIndex as well. Since the partial string selection is a form of label slicing, the endpoints will be included. This would include matching times on an included date. Here’s an example:

In [74]:
dft = pd.DataFrame(np.random.randn(100000,1), columns=['A'], 
                   index=pd.date_range('20130101', periods=100000, freq='T'))

In [75]:
dft.shape

(100000, 1)

In [76]:
dft

,A
2013-01-01 00:00:00,-0.397880
2013-01-01 00:01:00,-1.259224
2013-01-01 00:02:00,-0.688879
2013-01-01 00:03:00,0.802630
2013-01-01 00:04:00,0.272391
2013-01-01 00:05:00,-0.969176
2013-01-01 00:06:00,0.871968
2013-01-01 00:07:00,-1.446359
2013-01-01 00:08:00,-0.536481
2013-01-01 00:09:00,0.197921


In [77]:
dft['2013']

,A
2013-01-01 00:00:00,-0.397880
2013-01-01 00:01:00,-1.259224
2013-01-01 00:02:00,-0.688879
2013-01-01 00:03:00,0.802630
2013-01-01 00:04:00,0.272391
2013-01-01 00:05:00,-0.969176
2013-01-01 00:06:00,0.871968
2013-01-01 00:07:00,-1.446359
2013-01-01 00:08:00,-0.536481
2013-01-01 00:09:00,0.197921


This starts on the very first time in the month, and includes the last date & time for the month:

In [78]:
dft['2013-1':'2013-2']

,A
2013-01-01 00:00:00,-0.397880
2013-01-01 00:01:00,-1.259224
2013-01-01 00:02:00,-0.688879
2013-01-01 00:03:00,0.802630
2013-01-01 00:04:00,0.272391
2013-01-01 00:05:00,-0.969176
2013-01-01 00:06:00,0.871968
2013-01-01 00:07:00,-1.446359
2013-01-01 00:08:00,-0.536481
2013-01-01 00:09:00,0.197921


This specifies a stop time that includes all of the times on the last day:

In [79]:
dft['2013-1':'2013-2-28']

,A
2013-01-01 00:00:00,-0.397880
2013-01-01 00:01:00,-1.259224
2013-01-01 00:02:00,-0.688879
2013-01-01 00:03:00,0.802630
2013-01-01 00:04:00,0.272391
2013-01-01 00:05:00,-0.969176
2013-01-01 00:06:00,0.871968
2013-01-01 00:07:00,-1.446359
2013-01-01 00:08:00,-0.536481
2013-01-01 00:09:00,0.197921


This specifies an exact stop time (and is not the same as the above):

In [80]:
dft['2013-1':'2013-2-28 00:00:00']

,A
2013-01-01 00:00:00,-0.397880
2013-01-01 00:01:00,-1.259224
2013-01-01 00:02:00,-0.688879
2013-01-01 00:03:00,0.802630
2013-01-01 00:04:00,0.272391
2013-01-01 00:05:00,-0.969176
2013-01-01 00:06:00,0.871968
2013-01-01 00:07:00,-1.446359
2013-01-01 00:08:00,-0.536481
2013-01-01 00:09:00,0.197921


We are stopping on the included end-point as it is part of the index:

In [81]:
dft['2013-1-15':'2013-1-15 12:30:00']

,A
2013-01-15 00:00:00,0.379735
2013-01-15 00:01:00,-0.898181
2013-01-15 00:02:00,-0.310935
2013-01-15 00:03:00,1.069495
2013-01-15 00:04:00,1.050672
2013-01-15 00:05:00,0.233499
2013-01-15 00:06:00,1.031788
2013-01-15 00:07:00,1.366767
2013-01-15 00:08:00,-0.271464
2013-01-15 00:09:00,1.324011


To select a single row, use .loc:

In [82]:
dft.loc['2013-1-15 12:30:00']

A   -3.286898
Name: 2013-01-15 12:30:00, dtype: float64

### Datetime indexing

Indexing a DateTimeIndex with a partial string depends on the “accuracy” of the period, in other words how specific the interval is in relation to the frequency of the index. In contrast, indexing with datetime objects is exact, because the objects have exact meaning. These also follow the semantics of including both endpoints.

These datetime objects are specific hours, minutes, and seconds even though they were not explicitly specified (they are 0).

In [83]:
dft[pd.datetime(2013, 1, 1):pd.datetime(2013, 2, 28)]

,A
2013-01-01 00:00:00,-0.397880
2013-01-01 00:01:00,-1.259224
2013-01-01 00:02:00,-0.688879
2013-01-01 00:03:00,0.802630
2013-01-01 00:04:00,0.272391
2013-01-01 00:05:00,-0.969176
2013-01-01 00:06:00,0.871968
2013-01-01 00:07:00,-1.446359
2013-01-01 00:08:00,-0.536481
2013-01-01 00:09:00,0.197921


With no defaults:

In [84]:
dft[pd.datetime(2013, 1, 1, 10, 12, 0):pd.datetime(2013, 2, 28, 10, 12, 0)]

,A
2013-01-01 10:12:00,0.940664
2013-01-01 10:13:00,-0.252741
2013-01-01 10:14:00,1.206421
2013-01-01 10:15:00,-1.173520
2013-01-01 10:16:00,-0.909358
2013-01-01 10:17:00,1.158369
2013-01-01 10:18:00,-0.992256
2013-01-01 10:19:00,0.051982
2013-01-01 10:20:00,-0.254945
2013-01-01 10:21:00,1.389627


### Truncating & fancy indexing

A truncate convenience function is provided that is equivalent to slicing:

In [85]:
ts.truncate(before='10/31/2011', after='12/31/2011')

2011-10-31    0.146560
2011-11-30   -0.466351
2011-12-30    0.356223
Freq: BM, dtype: float64

Even complicated fancy indexing that breaks the DatetimeIndex’s frequency regularity will result in a DatetimeIndex (but frequency is lost):

In [86]:
ts[[0, 2, 6]].index

DatetimeIndex(['2011-01-31', '2011-03-31', '2011-07-29'], dtype='datetime64[ns]', freq=None)

### Time/Date Components

There are several time/date properties that one can access from Timestamp or a collection of timestamps like a DateTimeIndex.


Property | Description
---------|------------
year | The year of the datetime
month | The month of the datetime
day | The days of the datetime
hour | The hour of the datetime
minute | The minutes of the datetime
second | The seconds of the datetime
microsecond | The microseconds of the datetime
nanosecond | The nanoseconds of the datetime
date | Returns datetime.date
time | Returns datetime.time
dayofyear | The ordinal day of year
weekofyear | The week ordinal of the year
week | The week ordinal of the year
dayofweek | The day of the week with Monday=0, Sunday=6
weekday | The day of the week with Monday=0, Sunday=6
quarter | Quarter of the date: Jan=Mar = 1, Apr-Jun = 2, etc.
days_in_month | The number of days in the month of the datetime
is_month_start | Logical indicating if first day of month (defined by frequency)
is_month_end | Logical indicating if last day of month (defined by frequency)
is_quarter_start | Logical indicating if first day of quarter (defined by frequency)
is_quarter_end | Logical indicating if last day of quarter (defined by frequency)
is_year_start | Logical indicating if first day of year (defined by frequency)
is_year_end | Logical indicating if last day of year (defined by frequency)

Furthermore, if you have a Series with datetimelike values, then you can access these properties via the .dt accessor, see the docs.

In [87]:
ts

2011-01-31   -0.571817
2011-02-28   -0.603299
2011-03-31   -1.339389
2011-04-29   -1.689653
2011-05-31   -0.199327
2011-06-30    0.257773
2011-07-29    1.828821
2011-08-31   -1.001002
2011-09-30   -2.091691
2011-10-31    0.146560
2011-11-30   -0.466351
2011-12-30    0.356223
Freq: BM, dtype: float64

In [88]:
ts.index.month

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64')

## DateOffset objects

In the preceding examples, we created DatetimeIndex objects at various frequencies by passing in frequency strings like 'M', 'W', and 'BM' to the freq keyword. Under the hood, these frequency strings are being translated into an instance of pandas DateOffset, which represents a regular frequency increment. Specific offset logic like “month”, “business day”, or “one hour” is represented in its various subclasses.

Class name | Description
-----------|------------
DateOffset | Generic offset class, defaults to 1 calendar day
BDay | business day (weekday)
CDay | custom business day (experimental)
Week | one week, optionally anchored on a day of the week
WeekOfMonth | the x-th day of the y-th week of each month
LastWeekOfMonth | the x-th day of the last week of each month
MonthEnd | calendar month end
MonthBegin | calendar month begin
BMonthEnd | business month end
BMonthBegin | business month begin
CBMonthEnd | custom business month end
CBMonthBegin | custom business month begin
QuarterEnd | calendar quarter end
QuarterBegin | calendar quarter begin
BQuarterEnd | business quarter end
BQuarterBegin | business quarter begin
FY5253Quarter | retail (aka 52-53 week) quarter
YearEnd | calendar year end
YearBegin | calendar year begin
BYearEnd | business year end
BYearBegin | business year begin
FY5253 | retail (aka 52-53 week) year
BusinessHour | business hour
Hour | one hour
Minute | one minute
Second | one second
Milli | one millisecond
Micro | one microsecond
Nano | one nanosecond

Basic function of DateOffset:

In [89]:
d = pd.datetime(2018, 2, 27, 9, 0)

In [90]:
d

datetime.datetime(2018, 2, 27, 9, 0)

In [91]:
pd.Timestamp(d)

Timestamp('2018-02-27 09:00:00')

In [92]:
d + pd.tseries.offsets.DateOffset(months=2, days=1)

Timestamp('2018-04-28 09:00:00')

The key features of a DateOffset object are:

* it can be added / subtracted to/from a datetime object to obtain a shifted date
* it can be multiplied by an integer (positive or negative) so that the increment will be applied multiple times
* it has rollforward and rollback methods for moving a date forward or backward to the next or previous “offset date”

Subclasses of DateOffset define the apply function which dictates custom date increment logic, such as adding business days:

    class BDay(DateOffset):
        """DateOffset increments between business days"""
        def apply(self, other):
            ...

In [93]:
d - 5 * pd.tseries.offsets.BDay()

Timestamp('2018-02-20 09:00:00')

In [94]:
d + pd.tseries.offsets.BMonthEnd()

Timestamp('2018-02-28 09:00:00')

The rollforward and rollback methods do exactly what you would expect:

In [95]:
d

datetime.datetime(2018, 2, 27, 9, 0)

In [96]:
offset = pd.tseries.offsets.BMonthEnd()

In [97]:
offset.rollforward(d)

Timestamp('2018-02-28 09:00:00')

In [98]:
offset.rollback(d)

Timestamp('2018-01-31 09:00:00')

If you expect to use these functions, explore the pandas.tseries.offsets module and the various docstrings for the classes.

These operations (apply, rollforward and rollback) preserves time (hour, minute, etc) information by default. To reset time, use normalize=True keyword when creating the offset instance. If normalize=True, result is normalized after the function is applied.

In [99]:
day = pd.tseries.offsets.Day()

In [100]:
day.apply(pd.Timestamp('2014-01-01 09:00'))

Timestamp('2014-01-02 09:00:00')

In [101]:
day = pd.tseries.offsets.Day(normalize=True)

In [102]:
day.apply(pd.Timestamp('2014-01-01 09:00'))

Timestamp('2014-01-02 00:00:00')

In [103]:
hour = pd.tseries.offsets.Hour()

In [104]:
hour.apply(pd.Timestamp('2014-01-01 22:00'))

Timestamp('2014-01-01 23:00:00')

In [105]:
pd.Timestamp('2014-01-01 23:00:00')

Timestamp('2014-01-01 23:00:00')

In [106]:
hour = pd.tseries.offsets.Hour(normalize=True)

In [107]:
hour.apply(pd.Timestamp('2014-01-01 22:00'))

Timestamp('2014-01-01 00:00:00')

In [108]:
hour.apply(pd.Timestamp('2014-01-01 23:00'))

Timestamp('2014-01-02 00:00:00')

### Parametric offsets

Some of the offsets can be “parameterized” when created to result in different behaviors. For example, the Week offset for generating weekly data accepts a weekday parameter which results in the generated dates always lying on a particular day of the week:

In [109]:
d

datetime.datetime(2018, 2, 27, 9, 0)

In [110]:
d + pd.tseries.offsets.Week()

Timestamp('2018-03-06 09:00:00')

In [111]:
d + pd.tseries.offsets.Week(weekday=4)

Timestamp('2018-03-02 09:00:00')

In [112]:
(d + pd.tseries.offsets.Week(weekday=4)).weekday()

4

In [113]:
d - pd.tseries.offsets.Week()

Timestamp('2018-02-20 09:00:00')

normalize option will be effective for addition and subtraction.

In [114]:
d + pd.tseries.offsets.Week(normalize=True)

Timestamp('2018-03-06 00:00:00')

In [115]:
d - pd.tseries.offsets.Week(normalize=True)

Timestamp('2018-02-20 00:00:00')

Another example is parameterizing YearEnd with the specific ending month:

In [116]:
d + pd.tseries.offsets.YearEnd()

Timestamp('2018-12-31 09:00:00')

In [117]:
d + pd.tseries.offsets.YearEnd(month=6)

Timestamp('2018-06-30 09:00:00')

### Using offsets with Series / DatetimeIndex

Offsets can be used with either a Series or DatetimeIndex to apply the offset to each element.

In [118]:
rng = pd.date_range('2012-01-01', '2012-01-03')

In [119]:
s = pd.Series(rng)

In [120]:
rng

DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03'], dtype='datetime64[ns]', freq='D')

In [121]:
rng + pd.tseries.offsets.DateOffset(months=2)

DatetimeIndex(['2012-03-01', '2012-03-02', '2012-03-03'], dtype='datetime64[ns]', freq='D')

In [122]:
s + pd.tseries.offsets.DateOffset(months=2)

0   2012-03-01
1   2012-03-02
2   2012-03-03
dtype: datetime64[ns]

In [123]:
s - pd.tseries.offsets.DateOffset(months=2)

0   2011-11-01
1   2011-11-02
2   2011-11-03
dtype: datetime64[ns]

If the offset class maps directly to a Timedelta (Day, Hour, Minute, Second, Micro, Milli, Nano) it can be used exactly like a Timedelta - see the Timedelta section for more examples.

In [124]:
s - pd.tseries.offsets.Day(2)

0   2011-12-30
1   2011-12-31
2   2012-01-01
dtype: datetime64[ns]

In [125]:
td = s - pd.Series(pd.date_range('2011-12-29', '2011-12-31'))

In [126]:
td

0   3 days
1   3 days
2   3 days
dtype: timedelta64[ns]

In [127]:
td + pd.tseries.offsets.Minute(15)

0   3 days 00:15:00
1   3 days 00:15:00
2   3 days 00:15:00
dtype: timedelta64[ns]

Note that some offsets (such as BQuarterEnd) do not have a vectorized implementation. They can still be used but may calculate signficantly slower and will raise a PerformanceWarning.

In [128]:
rng + pd.tseries.offsets.BQuarterEnd()

/Users/luke.thompson/miniconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexes/datetimes.py:840: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  "or DatetimeIndex", PerformanceWarning)


DatetimeIndex(['2012-03-30', '2012-03-30', '2012-03-30'], dtype='datetime64[ns]', freq=None)

### Offset Aliases

A number of string aliases are given to useful common time series frequencies. We will refer to these aliases as offset aliases (referred to as time rules prior to v0.8.0).

Alias | Description
------|------------
B | business day frequency
C | custom business day frequency (experimental)
D | calendar day frequency
W | weekly frequency
M | month end frequency
BM | business month end frequency
CBM | custom business month end frequency
MS | month start frequency
BMS | business month start frequency
CBMS | custom business month start frequency
Q | quarter end frequency
BQ | business quarter endfrequency
QS | quarter start frequency
BQS | business quarter start frequency
A | year end frequency
BA | business year end frequency
AS | year start frequency
BAS | business year start frequency
BH | business hour frequency
H | hourly frequency
T, min | minutely frequency
S | secondly frequency
L, ms | milliseonds
U, us | microseconds
N | nanoseconds

### Combining Aliases

As we have seen previously, the alias and the offset instance are fungible in most functions:

In [129]:
pd.date_range(start, periods=5, freq='B')

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07'],
              dtype='datetime64[ns]', freq='B')

In [130]:
pd.date_range(start, periods=5, freq=pd.tseries.offsets.BDay())

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07'],
              dtype='datetime64[ns]', freq='B')

You can combine together day and intraday offsets:

In [131]:
pd.date_range(start, periods=10, freq='2h20min')

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 02:20:00',
               '2011-01-01 04:40:00', '2011-01-01 07:00:00',
               '2011-01-01 09:20:00', '2011-01-01 11:40:00',
               '2011-01-01 14:00:00', '2011-01-01 16:20:00',
               '2011-01-01 18:40:00', '2011-01-01 21:00:00'],
              dtype='datetime64[ns]', freq='140T')

In [132]:
pd.date_range(start, periods=10, freq='1D10U')

DatetimeIndex([       '2011-01-01 00:00:00', '2011-01-02 00:00:00.000010',
               '2011-01-03 00:00:00.000020', '2011-01-04 00:00:00.000030',
               '2011-01-05 00:00:00.000040', '2011-01-06 00:00:00.000050',
               '2011-01-07 00:00:00.000060', '2011-01-08 00:00:00.000070',
               '2011-01-09 00:00:00.000080', '2011-01-10 00:00:00.000090'],
              dtype='datetime64[ns]', freq='86400000010U')

## Time series-related instance methods

### Shifting / lagging

One may want to shift or lag the values in a time series back and forward in time. The method for this is shift, which is available on all of the pandas objects.

In [133]:
ts = pd.Series(['a', 'b', 'c', 'd', 'e'], index=pd.date_range(start='3/1/18', periods=5))

In [134]:
ts

2018-03-01    a
2018-03-02    b
2018-03-03    c
2018-03-04    d
2018-03-05    e
Freq: D, dtype: object

In [135]:
ts.shift(3)

2018-03-01    NaN
2018-03-02    NaN
2018-03-03    NaN
2018-03-04      a
2018-03-05      b
Freq: D, dtype: object

Notes: If freq is specified then the index values are shifted but the data
is not realigned. That is, use freq if you would like to extend the
index when shifting and preserve the original data.

In [136]:
ts.shift(3, freq='D')

2018-03-04    a
2018-03-05    b
2018-03-06    c
2018-03-07    d
2018-03-08    e
Freq: D, dtype: object

In [137]:
ts.shift(3, freq='M')

2018-05-31    a
2018-05-31    b
2018-05-31    c
2018-05-31    d
2018-05-31    e
dtype: object

The same thing can be accomplisehd with the tshift convenience method that changes all the dates in the index by a specified number of offsets, rather than changing the alignment of the data and the index:

In [138]:
ts.tshift(3)

2018-03-04    a
2018-03-05    b
2018-03-06    c
2018-03-07    d
2018-03-08    e
Freq: D, dtype: object

Note that with tshift (and with shift+freq), the leading entry is no longer NaN because the data is not being realigned.

### Frequency conversion

The primary function for changing frequencies is the asfreq function. For a DatetimeIndex, this is a convenient wrapper around reindex which generates a date_range and calls reindex.

In [139]:
dr = pd.date_range('1/1/2010', periods=3, freq='3B')

In [140]:
ts = pd.Series(np.random.randn(3), index=dr)

In [141]:
ts

2010-01-01   -0.879905
2010-01-06   -1.414049
2010-01-11    0.758031
Freq: 3B, dtype: float64

In [142]:
ts.asfreq('B')

2010-01-01   -0.879905
2010-01-04         NaN
2010-01-05         NaN
2010-01-06   -1.414049
2010-01-07         NaN
2010-01-08         NaN
2010-01-11    0.758031
Freq: B, dtype: float64

asfreq provides a further convenience so you can specify an interpolation method (e.g. 'pad', i.e. forward fill) for any gaps that may appear after the frequency conversion.

In [143]:
ts.asfreq(pd.tseries.offsets.BDay(), method='pad')

2010-01-01   -0.879905
2010-01-04   -0.879905
2010-01-05   -0.879905
2010-01-06   -1.414049
2010-01-07   -1.414049
2010-01-08   -1.414049
2010-01-11    0.758031
Freq: B, dtype: float64

### Filling forward / backward

Related to asfreq and reindex is the fillna function, which can be used with any Pandas object.

In [144]:
ts.asfreq('B').fillna(0)

2010-01-01   -0.879905
2010-01-04    0.000000
2010-01-05    0.000000
2010-01-06   -1.414049
2010-01-07    0.000000
2010-01-08    0.000000
2010-01-11    0.758031
Freq: B, dtype: float64

## Time Span Representation

Regular intervals of time are represented by Period objects in pandas while sequences of Period objects are collected in a PeriodIndex, which can be created with the convenience function period_range.

### Period

A Period represents a span of time (e.g., a day, a month, a quarter, etc). You can specify the span via freq keyword using a frequency alias like below. Because freq represents a span of Period, it cannot be negative like "-3D". ('A-DEC' is annual frequency, anchored end of December; same as 'A'.)

In [145]:
pd.Period('2012', freq='A')

Period('2012', 'A-DEC')

In [146]:
pd.Period('2012-1-1', freq='D')

Period('2012-01-01', 'D')

In [147]:
pd.Period('2012-1-1 19:00', freq='H')

Period('2012-01-01 19:00', 'H')

In [148]:
pd.Period('2012-1-1 19:00', freq='5H')

Period('2012-01-01 19:00', '5H')

Adding and subtracting integers from periods shifts the period by its own frequency. Arithmetic is not allowed between Period with different freq (span).

In [149]:
p = pd.Period('2012', freq='A')

In [150]:
p + 1

Period('2013', 'A-DEC')

In [151]:
p - 3

Period('2009', 'A-DEC')

In [152]:
p = pd.Period('2012-01', freq='2M')

In [153]:
p + 2

Period('2012-05', '2M')

In [154]:
p - 1

Period('2011-11', '2M')

If Period freq is daily or higher (D, H, T, S, L, U, N), offsets and timedelta-like can be added if the result can have the same freq.

In [155]:
p = pd.Period('2014-07-01 09:00', freq='H')

In [156]:
p + pd.tseries.offsets.Hour(2)

Period('2014-07-01 11:00', 'H')

In [157]:
p + pd.tseries.offsets.timedelta(minutes=120)

Period('2014-07-01 11:00', 'H')

In [158]:
p + np.timedelta64(7200, 's')

Period('2014-07-01 11:00', 'H')

If Period has other freqs, only the same offsets can be added.

In [159]:
p = pd.Period('2014-07', freq='M')

In [160]:
p + pd.tseries.offsets.MonthEnd(3)

Period('2014-10', 'M')

In [161]:
pd.Period('2012', freq='A') - pd.Period('2002', freq='A')

10

### Periods vs. Timestamps

Notice the difference between Timestamps and Periods in determining differences in times.

In [162]:
pd.datetime.today() - pd.Timestamp('1/1/1970')

Timedelta('17589 days 10:45:57.466213')

In [163]:
pd.Period(pd.datetime.today(), 'D') - pd.Period('1/1/1970', 'D')

17589

In [164]:
pd.Period(pd.datetime.today(), 'A') - pd.Period('1/1/1970', 'A')

48

In [165]:
t = pd.datetime.today() - pd.Timestamp('1/1/1970')

In [166]:
t.days

17589

### PeriodIndex and period_range

Regular sequences of Period objects can be collected in a PeriodIndex, which can be constructed using the period_range convenience function:

In [167]:
prng = pd.period_range('1/1/2011', '1/1/2012', freq='M')

In [168]:
prng

PeriodIndex(['2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
             '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
             '2012-01'],
            dtype='period[M]', freq='M')

The PeriodIndex constructor can also be used directly:

In [169]:
pd.PeriodIndex(['2011-1', '2011-2', '2011-3'], freq='M')

PeriodIndex(['2011-01', '2011-02', '2011-03'], dtype='period[M]', freq='M')

Passing multiplied frequency outputs a sequence of Period which has multiplied span.

In [170]:
pd.PeriodIndex(start='2014-01', freq='3M', periods=4)

PeriodIndex(['2014-01', '2014-04', '2014-07', '2014-10'], dtype='period[3M]', freq='3M')

Just like DatetimeIndex, a PeriodIndex can also be used to index pandas objects:

In [171]:
ps = pd.Series(np.random.randn(len(prng)), prng)

In [172]:
ps

2011-01    0.610397
2011-02    2.718729
2011-03    0.123990
2011-04    0.061996
2011-05   -1.098168
2011-06    0.351612
2011-07   -1.165559
2011-08   -0.055498
2011-09   -0.121368
2011-10   -0.684034
2011-11   -0.936657
2011-12   -0.925786
2012-01   -0.039955
Freq: M, dtype: float64

PeriodIndex supports addition and subtraction with the same rule as Period.

In [173]:
idx = pd.period_range('2014-07-01 09:00', periods=5, freq='H')

In [174]:
idx

PeriodIndex(['2014-07-01 09:00', '2014-07-01 10:00', '2014-07-01 11:00',
             '2014-07-01 12:00', '2014-07-01 13:00'],
            dtype='period[H]', freq='H')

In [175]:
idx + pd.tseries.offsets.Hour(2)

PeriodIndex(['2014-07-01 11:00', '2014-07-01 12:00', '2014-07-01 13:00',
             '2014-07-01 14:00', '2014-07-01 15:00'],
            dtype='period[H]', freq='H')

In [176]:
idx = pd.period_range('2014-07', periods=5, freq='M')

In [177]:
idx

PeriodIndex(['2014-07', '2014-08', '2014-09', '2014-10', '2014-11'], dtype='period[M]', freq='M')

In [178]:
idx + pd.tseries.offsets.MonthEnd(3)

PeriodIndex(['2014-10', '2014-11', '2014-12', '2015-01', '2015-02'], dtype='period[M]', freq='M')

### PeriodIndex partial string indexing

You can pass in dates and strings to Series and DataFrame with PeriodIndex, in the same manner as DatetimeIndex. For details, refer to DatetimeIndex Partial String Indexing.

In [179]:
ps

2011-01    0.610397
2011-02    2.718729
2011-03    0.123990
2011-04    0.061996
2011-05   -1.098168
2011-06    0.351612
2011-07   -1.165559
2011-08   -0.055498
2011-09   -0.121368
2011-10   -0.684034
2011-11   -0.936657
2011-12   -0.925786
2012-01   -0.039955
Freq: M, dtype: float64

In [180]:
ps['2011-01']

0.6103969193827244

In [181]:
ps[pd.datetime(2011, 12, 25):]

2011-12   -0.925786
2012-01   -0.039955
Freq: M, dtype: float64

In [182]:
ps['10/31/2011':'12/31/2011']

2011-10   -0.684034
2011-11   -0.936657
2011-12   -0.925786
Freq: M, dtype: float64

Passing a string representing a lower frequency than PeriodIndex returns partial sliced data.

In [183]:
ps['2011']

2011-01    0.610397
2011-02    2.718729
2011-03    0.123990
2011-04    0.061996
2011-05   -1.098168
2011-06    0.351612
2011-07   -1.165559
2011-08   -0.055498
2011-09   -0.121368
2011-10   -0.684034
2011-11   -0.936657
2011-12   -0.925786
Freq: M, dtype: float64

In [184]:
dfp = pd.DataFrame(np.random.randn(600,1), columns=['A'], index=pd.period_range('2013-01-01 9:00', periods=600, freq='T'))

In [185]:
dfp

,A
2013-01-01 09:00,-0.202472
2013-01-01 09:01,0.471613
2013-01-01 09:02,0.412381
2013-01-01 09:03,1.468281
2013-01-01 09:04,-0.029521
2013-01-01 09:05,-2.514410
2013-01-01 09:06,-0.728389
2013-01-01 09:07,0.361196
2013-01-01 09:08,-0.062308
2013-01-01 09:09,-0.150668


In [186]:
dfp['2013-01-01 10H']

,A
2013-01-01 10:00,-0.214422
2013-01-01 10:01,0.914302
2013-01-01 10:02,1.256289
2013-01-01 10:03,-0.670824
2013-01-01 10:04,0.425011
2013-01-01 10:05,-0.313596
2013-01-01 10:06,0.917931
2013-01-01 10:07,-0.570293
2013-01-01 10:08,0.255517
2013-01-01 10:09,-0.167031


As with DatetimeIndex, the endpoints will be included in the result. The example below slices data starting from 10:00 to 11:59.

In [187]:
dfp['2013-01-01 10H':'2013-01-01 11H']

,A
2013-01-01 10:00,-0.214422
2013-01-01 10:01,0.914302
2013-01-01 10:02,1.256289
2013-01-01 10:03,-0.670824
2013-01-01 10:04,0.425011
2013-01-01 10:05,-0.313596
2013-01-01 10:06,0.917931
2013-01-01 10:07,-0.570293
2013-01-01 10:08,0.255517
2013-01-01 10:09,-0.167031


### Frequency conversion and resampling with PeriodIndex

The frequency of Period and PeriodIndex can be converted via the asfreq method. Let’s start with the fiscal year 2011, ending in December:

In [188]:
p = pd.Period('2011', freq='A-DEC')

In [189]:
p

Period('2011', 'A-DEC')

We can convert it to a monthly frequency. Using the how parameter, we can specify whether to return the starting or ending month:

In [190]:
p.asfreq('M', how='start')

Period('2011-01', 'M')

In [191]:
p.asfreq('M', how='end')

Period('2011-12', 'M')

## Converting between representations

Timestamped data can be converted to PeriodIndex-ed data using to_period and vice-versa using to_timestamp:

In [192]:
rng = pd.date_range('1/1/2012', periods=5, freq='M')

In [193]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [194]:
ts

2012-01-31   -0.355537
2012-02-29   -0.696925
2012-03-31   -0.613224
2012-04-30    1.052651
2012-05-31   -1.458326
Freq: M, dtype: float64

In [195]:
ps = ts.to_period()

In [196]:
ps

2012-01   -0.355537
2012-02   -0.696925
2012-03   -0.613224
2012-04    1.052651
2012-05   -1.458326
Freq: M, dtype: float64

In [197]:
ps.to_timestamp()

2012-01-01   -0.355537
2012-02-01   -0.696925
2012-03-01   -0.613224
2012-04-01    1.052651
2012-05-01   -1.458326
Freq: MS, dtype: float64

In [198]:
ps.to_timestamp('D', how='start')

2012-01-01   -0.355537
2012-02-01   -0.696925
2012-03-01   -0.613224
2012-04-01    1.052651
2012-05-01   -1.458326
Freq: MS, dtype: float64

In [199]:
ps.to_timestamp('D', how='end')

2012-01-31   -0.355537
2012-02-29   -0.696925
2012-03-31   -0.613224
2012-04-30    1.052651
2012-05-31   -1.458326
Freq: M, dtype: float64

## Time zone handling

Pandas provides rich support for working with timestamps in different time zones using pytz and dateutil libraries. dateutil support is new in 0.14.1 and currently only supported for fixed offset and tzfile zones. The default library is pytz. Support for dateutil is provided for compatibility with other applications e.g. if you use dateutil in other python packages.

### Working with time zones

By default, pandas objects are time zone unaware:

In [200]:
rng = pd.date_range('3/6/2012 00:00', periods=15, freq='D')

In [201]:
rng.tz is None

True

To supply the time zone, you can use the tz keyword to date_range and other functions. There are two options for time zone formats:

* pytz - pytz brings the Olson tz database into Python. This library allows accurate and cross platform timezone calculations using Python 2.4 or higher. You can find a list of common (and less common) time zones using from pytz import common_timezones, all_timezones. For UTC, there is a special case tzutc.
* dateutil - The dateutil module provides powerful extensions to the datetime module available in the Python standard library. It uses the OS timezones so there isn’t a fixed list available. For common zones, the names are the same as pytz. Dateutil time zone strings are distinguished from pytz time zones by starting with "dateutil/".

In [202]:
# pytz
rng_pytz = pd.date_range('3/6/2012 00:00', periods=10, freq='D', tz='Europe/London')

In [203]:
rng_pytz.tz

<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>

In [204]:
# dateutil
rng_dateutil = pd.date_range('3/6/2012 00:00', periods=10, freq='D', tz='dateutil/Europe/London')

In [205]:
rng_dateutil.tz

tzfile('/usr/share/zoneinfo/Europe/London')

In [206]:
# dateutil - utc special case
rng_utc = pd.date_range('3/6/2012 00:00', periods=10, freq='D', tz=dateutil.tz.tzutc())

In [207]:
rng_utc.tz

tzutc()

Note that the UTC timezone is a special case in dateutil and should be constructed explicitly as an instance of dateutil.tz.tzutc. You can also construct other timezones explicitly first, which gives you more control over which time zone is used:

In [208]:
# pytz
tz_pytz = pytz.timezone('Europe/London')

In [209]:
rng_pytz = pd.date_range('3/6/2012 00:00', periods=10, freq='D', tz=tz_pytz)

In [210]:
rng_pytz.tz == tz_pytz

True

In [211]:
# dateutil
tz_dateutil = dateutil.tz.gettz('Europe/London')

In [212]:
rng_dateutil = pd.date_range('3/6/2012 00:00', periods=10, freq='D', tz=tz_dateutil)

In [213]:
rng_dateutil.tz == tz_dateutil

True

Timestamps, like Python’s datetime.datetime object can be either time zone naive or time zone aware. Naive time series and DatetimeIndex objects can be localized using tz_localize:

In [214]:
ts = pd.Series(np.random.randn(len(rng)), rng)

In [215]:
ts

2012-03-06    1.391462
2012-03-07   -0.326334
2012-03-08   -0.101728
2012-03-09    0.443825
2012-03-10   -1.816127
2012-03-11   -0.133840
2012-03-12    0.840769
2012-03-13    1.498535
2012-03-14   -0.079269
2012-03-15   -0.244387
2012-03-16    0.187639
2012-03-17   -0.737377
2012-03-18   -2.282972
2012-03-19   -0.619518
2012-03-20    1.777953
Freq: D, dtype: float64

In [216]:
ts_utc = ts.tz_localize('UTC')

In [217]:
ts_utc

2012-03-06 00:00:00+00:00    1.391462
2012-03-07 00:00:00+00:00   -0.326334
2012-03-08 00:00:00+00:00   -0.101728
2012-03-09 00:00:00+00:00    0.443825
2012-03-10 00:00:00+00:00   -1.816127
2012-03-11 00:00:00+00:00   -0.133840
2012-03-12 00:00:00+00:00    0.840769
2012-03-13 00:00:00+00:00    1.498535
2012-03-14 00:00:00+00:00   -0.079269
2012-03-15 00:00:00+00:00   -0.244387
2012-03-16 00:00:00+00:00    0.187639
2012-03-17 00:00:00+00:00   -0.737377
2012-03-18 00:00:00+00:00   -2.282972
2012-03-19 00:00:00+00:00   -0.619518
2012-03-20 00:00:00+00:00    1.777953
Freq: D, dtype: float64

Again, you can explicitly construct the timezone object first. You can use the tz_convert method to convert pandas objects to convert tz-aware data to another time zone:

In [218]:
ts_utc.tz_convert('US/Eastern')

2012-03-05 19:00:00-05:00    1.391462
2012-03-06 19:00:00-05:00   -0.326334
2012-03-07 19:00:00-05:00   -0.101728
2012-03-08 19:00:00-05:00    0.443825
2012-03-09 19:00:00-05:00   -1.816127
2012-03-10 19:00:00-05:00   -0.133840
2012-03-11 20:00:00-04:00    0.840769
2012-03-12 20:00:00-04:00    1.498535
2012-03-13 20:00:00-04:00   -0.079269
2012-03-14 20:00:00-04:00   -0.244387
2012-03-15 20:00:00-04:00    0.187639
2012-03-16 20:00:00-04:00   -0.737377
2012-03-17 20:00:00-04:00   -2.282972
2012-03-18 20:00:00-04:00   -0.619518
2012-03-19 20:00:00-04:00    1.777953
Freq: D, dtype: float64

Warnings:

* Be wary of conversions between libraries. For some zones pytz and dateutil have different definitions of the zone. This is more of a problem for unusual timezones than for ‘standard’ zones like US/Eastern.
* Be aware that a timezone definition across versions of timezone libraries may not be considered equal. This may cause problems when working with stored data that is localized using one version and operated on with a different version. See here for how to handle such a situation.
* It is incorrect to pass a timezone directly into the datetime.datetime constructor (e.g., datetime.datetime(2011, 1, 1, tz=timezone('US/Eastern')). Instead, the datetime needs to be localized using the the localize method on the timezone.

Under the hood, all timestamps are stored in UTC. Scalar values from a DatetimeIndex with a time zone will have their fields (day, hour, minute) localized to the time zone. However, timestamps with the same UTC value are still considered to be equal even if they are in different time zones:

In [219]:
rng_eastern = rng_utc.tz_convert('US/Eastern')

In [220]:
rng_berlin = rng_utc.tz_convert('Europe/Berlin')

In [221]:
rng_eastern[5]

Timestamp('2012-03-10 19:00:00-0500', tz='US/Eastern', freq='D')

In [222]:
rng_berlin[5]

Timestamp('2012-03-11 01:00:00+0100', tz='Europe/Berlin', freq='D')

In [223]:
rng_eastern[5] == rng_berlin[5]

True

Like Series, DataFrame, and DatetimeIndex, Timestamps can be converted to other time zones using tz_convert:

In [224]:
rng_eastern[5]

Timestamp('2012-03-10 19:00:00-0500', tz='US/Eastern', freq='D')

In [225]:
rng_berlin[5]

Timestamp('2012-03-11 01:00:00+0100', tz='Europe/Berlin', freq='D')

In [226]:
rng_eastern[5].tz_convert('Europe/Berlin')

Timestamp('2012-03-11 01:00:00+0100', tz='Europe/Berlin')

Localization of Timestamps functions just like DatetimeIndex and Series:

In [227]:
rng[5]

Timestamp('2012-03-11 00:00:00', freq='D')

In [228]:
rng[5].tz_localize('Asia/Shanghai')

Timestamp('2012-03-11 00:00:00+0800', tz='Asia/Shanghai')

Operations between Series in different time zones will yield UTC Series, aligning the data on the UTC timestamps:

In [229]:
eastern = ts_utc.tz_convert('US/Eastern')

In [230]:
berlin = ts_utc.tz_convert('Europe/Berlin')

In [231]:
result = eastern + berlin

In [232]:
result

2012-03-06 00:00:00+00:00    2.782924
2012-03-07 00:00:00+00:00   -0.652667
2012-03-08 00:00:00+00:00   -0.203455
2012-03-09 00:00:00+00:00    0.887650
2012-03-10 00:00:00+00:00   -3.632254
2012-03-11 00:00:00+00:00   -0.267680
2012-03-12 00:00:00+00:00    1.681539
2012-03-13 00:00:00+00:00    2.997070
2012-03-14 00:00:00+00:00   -0.158539
2012-03-15 00:00:00+00:00   -0.488773
2012-03-16 00:00:00+00:00    0.375278
2012-03-17 00:00:00+00:00   -1.474753
2012-03-18 00:00:00+00:00   -4.565945
2012-03-19 00:00:00+00:00   -1.239035
2012-03-20 00:00:00+00:00    3.555907
Freq: D, dtype: float64

In [233]:
result.index

DatetimeIndex(['2012-03-06', '2012-03-07', '2012-03-08', '2012-03-09',
               '2012-03-10', '2012-03-11', '2012-03-12', '2012-03-13',
               '2012-03-14', '2012-03-15', '2012-03-16', '2012-03-17',
               '2012-03-18', '2012-03-19', '2012-03-20'],
              dtype='datetime64[ns, UTC]', freq='D')

### Other time zones

List all the time zones available from pytz:

In [234]:
for tz in pytz.all_timezones:
    print(tz)

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
Africa/Asmera
Africa/Bamako
Africa/Bangui
Africa/Banjul
Africa/Bissau
Africa/Blantyre
Africa/Brazzaville
Africa/Bujumbura
Africa/Cairo
Africa/Casablanca
Africa/Ceuta
Africa/Conakry
Africa/Dakar
Africa/Dar_es_Salaam
Africa/Djibouti
Africa/Douala
Africa/El_Aaiun
Africa/Freetown
Africa/Gaborone
Africa/Harare
Africa/Johannesburg
Africa/Juba
Africa/Kampala
Africa/Khartoum
Africa/Kigali
Africa/Kinshasa
Africa/Lagos
Africa/Libreville
Africa/Lome
Africa/Luanda
Africa/Lubumbashi
Africa/Lusaka
Africa/Malabo
Africa/Maputo
Africa/Maseru
Africa/Mbabane
Africa/Mogadishu
Africa/Monrovia
Africa/Nairobi
Africa/Ndjamena
Africa/Niamey
Africa/Nouakchott
Africa/Ouagadougou
Africa/Porto-Novo
Africa/Sao_Tome
Africa/Timbuktu
Africa/Tripoli
Africa/Tunis
Africa/Windhoek
America/Adak
America/Anchorage
America/Anguilla
America/Antigua
America/Araguaina
America/Argentina/Buenos_Aires
America/Argentina/Catamarca
America/Argentina/ComodRivad

For example, Berlin is GMT+1 (flip the sign):

In [235]:
rng.tz_localize('Europe/Berlin')

DatetimeIndex(['2012-03-06 00:00:00+01:00', '2012-03-07 00:00:00+01:00',
               '2012-03-08 00:00:00+01:00', '2012-03-09 00:00:00+01:00',
               '2012-03-10 00:00:00+01:00', '2012-03-11 00:00:00+01:00',
               '2012-03-12 00:00:00+01:00', '2012-03-13 00:00:00+01:00',
               '2012-03-14 00:00:00+01:00', '2012-03-15 00:00:00+01:00',
               '2012-03-16 00:00:00+01:00', '2012-03-17 00:00:00+01:00',
               '2012-03-18 00:00:00+01:00', '2012-03-19 00:00:00+01:00',
               '2012-03-20 00:00:00+01:00'],
              dtype='datetime64[ns, Europe/Berlin]', freq='D')

In [236]:
rng.tz_localize('Etc/GMT-1')

DatetimeIndex(['2012-03-06 00:00:00+01:00', '2012-03-07 00:00:00+01:00',
               '2012-03-08 00:00:00+01:00', '2012-03-09 00:00:00+01:00',
               '2012-03-10 00:00:00+01:00', '2012-03-11 00:00:00+01:00',
               '2012-03-12 00:00:00+01:00', '2012-03-13 00:00:00+01:00',
               '2012-03-14 00:00:00+01:00', '2012-03-15 00:00:00+01:00',
               '2012-03-16 00:00:00+01:00', '2012-03-17 00:00:00+01:00',
               '2012-03-18 00:00:00+01:00', '2012-03-19 00:00:00+01:00',
               '2012-03-20 00:00:00+01:00'],
              dtype='datetime64[ns, Etc/GMT-1]', freq='D')

We can check timezones by converting the current time 'now' to another time zone:

In [237]:
pd.Timestamp('now', tz='US/Eastern')

Timestamp('2018-02-27 13:45:58.756891-0500', tz='US/Eastern')

In [238]:
pd.Timestamp('now', tz='UTC')

Timestamp('2018-02-27 18:45:58.769158+0000', tz='UTC')

In [239]:
pd.Timestamp('now', tz='Asia/Riyadh')

Timestamp('2018-02-27 21:45:58.788558+0300', tz='Asia/Riyadh')

In [240]:
pd.Timestamp('now', tz='Etc/GMT-3')

Timestamp('2018-02-27 21:45:58.812796+0300', tz='Etc/GMT-3')

### Example: Day of week corresponding to a future date

In [241]:
def future_day_of_week(date, years_offset):
    # dictionary to rename day of week
    day_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
                4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    # covert date to timestamp
    ts = pd.Timestamp(date)
    # offset date by X years
    ts_offset = ts + pd.tseries.offsets.DateOffset(years=years_offset)
    # print with strftime to make date pretty
    print('%s in %s years will be %s which is a %s.' % (
            ts.strftime('%Y-%m-%d'), 
            years_offset, 
            ts_offset.strftime('%Y-%m-%d'), 
            day_dict[ts_offset.dayofweek]))

In [242]:
future_day_of_week('1/1/00', 21)

2000-01-01 in 21 years will be 2021-01-01 which is a Friday.


In [243]:
# days from today until a given day
pd.Period('1/1/21', 'D') - pd.Period(pd.datetime.today(), 'D')

1039